In [36]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV, train_test_split
# from xgboost import XGBClassifier
from sklearn.metrics import roc_curve
from sklearn.impute import KNNImputer

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Covid-tweets/training_data.csv'
file_key_2 = 'Covid-tweets/test_data.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)

In [5]:
train.head()

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us


In [7]:
train['country'].value_counts()

us             40000
uk             40000
canada         40000
australia      40000
ireland        40000
new_zealand    40000
Name: country, dtype: int64

In [8]:
train['is_quote'].value_counts()

False               109482
FALSE                89795
True                 21590
TRUE                 19132
Colin o'donoghue         1
Name: is_quote, dtype: int64

In [13]:
sum(train['is_quote'].isna())

0

In [15]:
train[train['is_quote'] == "Colin o'donoghue"]

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country
162661,0,NYCTogether,Colin o'donoghue,ireland,1498,ireland


In [11]:
train.shape

(240000, 6)

In [6]:
test.head()

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,Id
0,"Ethical investing is not optional anymore, say...",NaN,False,False,covid19,0
1,#COVID19 | Suite à la conférence de presse du ...,NaN,False,True,COVID19,1
2,"Yesterday, I had a live discussion with @Steve...",NaN,False,True,COVID19,2
3,Nepal - #Coronavirus cases up 24% in a week. D...,NaN,False,True,Coronavirus,3
4,American economy jumped up a % big news story...,NaN,False,False,LysolAndCloroxSales,4


# Feature Engineering 

In [37]:
train['is_quote'] = train['is_quote'].str.lower()
train['is_quote'] = np.where(train['is_quote'] == 'false', 0, 1)

test['is_quote'] = np.where(test['is_quote'] == False, 0, 1)

In [38]:
train['is_quote'].value_counts()

1    150205
0     89795
Name: is_quote, dtype: int64

In [39]:
test['is_quote'].value_counts()

0    49816
1    10184
Name: is_quote, dtype: int64